In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 02:20:19,623 INFO: Initializing external client
2025-03-04 02:20:19,623 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 02:20:20,413 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-03 07:20:19.622142+00:00 to 2025-03-04 06:20:19.622142+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.63s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 08:00:00+00:00,2,0
1,2025-02-03 09:00:00+00:00,2,0
2,2025-02-03 10:00:00+00:00,2,0
3,2025-02-03 11:00:00+00:00,2,0
4,2025-02-03 12:00:00+00:00,2,0
...,...,...,...
174189,2025-03-04 01:00:00+00:00,263,6
174190,2025-03-04 02:00:00+00:00,263,6
174191,2025-03-04 03:00:00+00:00,263,3
174192,2025-03-04 04:00:00+00:00,263,8


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,3,5,1,...,0,1,0,1,0,0,0,16,255,2025-02-11 02:00:00
1,0,0,1,0,0,0,6,2,0,0,...,2,0,1,0,0,1,0,1,112,2025-02-13 12:00:00
2,2,1,0,0,1,1,0,1,0,1,...,0,0,0,1,0,0,2,0,218,2025-02-08 04:00:00
3,369,273,249,21,251,348,17,373,286,251,...,1,222,32,324,393,166,35,89,237,2025-02-08 19:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-23 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-02-05 16:00:00
247,0,0,2,0,0,4,0,1,0,0,...,3,1,2,0,0,0,2,0,71,2025-02-13 18:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-26 16:00:00
249,0,1,1,1,1,2,0,4,0,1,...,0,1,1,2,4,1,0,0,168,2025-02-15 03:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 02:20:30,635 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 02:20:30,636 INFO: Initializing external client
2025-03-04 02:20:30,636 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 02:20:31,355 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-03 07:20:30.635217+00:00 to 2025-03-04 06:20:30.635217+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.50s) 


In [12]:
current_date

Timestamp('2025-03-04 07:20:30.635217+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 08:00:00
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,3,2025-03-03 08:00:00
2,3,4,6,2,2,2,4,4,1,4,...,1,0,1,2,0,1,1,7,4,2025-03-03 08:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 08:00:00
4,2,2,1,2,4,2,2,4,1,1,...,0,0,0,1,1,1,2,2,7,2025-03-03 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,2,0,259,2025-03-03 08:00:00
247,2,1,0,0,5,2,4,3,3,1,...,0,0,0,0,0,1,5,0,260,2025-03-03 08:00:00
248,11,10,7,26,13,25,37,25,38,30,...,0,0,0,0,2,2,2,2,261,2025-03-03 08:00:00
249,149,139,98,76,87,70,71,102,81,98,...,5,1,1,2,9,16,69,139,262,2025-03-03 08:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 02:20:39,966 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 02:20:39,968 INFO: Initializing external client
2025-03-04 02:20:39,968 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 02:20:40,718 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,5.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,1.0
248,261,6.0
249,262,147.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 186, 239, 132, 162, 238, 262, 142, 161], dtype=int32)